In [4]:
import requests
import json
import pandas as pd
import numpy as np
class CDEC_flow:
	def __init__(self,station,sensors,dur_code,start,end):
		url = f'https://cdec.water.ca.gov/dynamicapp/req/JSONDataServlet?Stations={station}&SensorNums={sensors}&dur_code={dur_code}&Start={start}&End={end}'
		print(url)
		R = requests.get(url)
		J = json.loads(R.text)
		self.flow = pd.DataFrame(J)
		if dur_code == "H":
			self.flow = self.clean_hourly()
		elif dur_code == "D":
			self.flow = self.clean_daily()


	def clean_daily(self):
		df = self.flow
		df.index = pd.to_datetime(df['date'])
		df['value'] = df['value'].where(df['value']>-9990,np.nan)
		return df

	def clean_hourly(self):
		df = self.flow
		df.index = pd.to_datetime(df['date'])
		# ! excel export and json export are shifted by 1
		df = df.shift(1, freq='H')
		df['value'] = df['value'].where(df['value']>-9990,np.nan)
		return df

In [14]:
# USGS2

def get_USGS_flow(site,start_date, end_date):
	# start_time = start_date.format("YYYY-MM-DDTHH:mm:ss.SSSZZ")
	# end_time = end_date.format("YYYY-MM-DDTHH:mm:ss.SSSZZ")
	time_format = "YYYY-MM-DDTHH:mm:ss.SSSZZ"
	time_format = "YYYY-MM-DDTHH:mm:ss.SSS-08:00"

	start_time = start_date.floor('day').format(time_format)
	print(start_time)
	end_time = end_date.floor('day').format(time_format)
	# https://waterservices.usgs.gov/nwis/iv/?sites=11274000&parameterCd=00060&startDT=2023-01-13T10:51:58.023-08:00&endDT=2023-01-14T10:51:58.023-08:00&siteStatus=all&format=rdb
	# Newman station
	url = f"https://waterservices.usgs.gov/nwis/iv/?sites={site}&parameterCd=00060&startDT={start_time}&endDT={end_time}&siteStatus=all&format=rdb"
	df = pd.read_csv(url, skiprows=26, sep="\t")
	df = df.drop(index = [0])
	return df

# "https://waterservices.usgs.gov/nwis/iv/?sites=11274000&parameterCd=00060&startDT=2022-12-15T09:12:20.947-08:00&endDT=2023-01-14T09:12:20.947-08:00&siteStatus=all&format=rdb"

"2022-12-15T09:12:20.947-08:00"
"2023-01-14T09:12:20.947-08:00"
start_date = arrow.now().shift(days=-2)
end_date = start_date.shift(days=+1)


newman = get_USGS_flow(
	"11274000",
	start_date,
	end_date,
)
newman

2023-01-13T00:00:00.000-08:00


,agency_cd,site_no,datetime,tz_cd,15012_00060,15012_00060_cd
1,USGS,11274000,2023-01-13 00:00,PST,10200,P
2,USGS,11274000,2023-01-13 00:15,PST,10200,P
3,USGS,11274000,2023-01-13 00:30,PST,10300,P
4,USGS,11274000,2023-01-13 00:45,PST,10300,P
5,USGS,11274000,2023-01-13 01:00,PST,10300,P
...,...,...,...,...,...,...
93,USGS,11274000,2023-01-13 23:00,PST,11800,P
94,USGS,11274000,2023-01-13 23:15,PST,11800,P
95,USGS,11274000,2023-01-13 23:30,PST,11800,P
96,USGS,11274000,2023-01-13 23:45,PST,11800,P


In [74]:
# class USGS:


# f"https://waterdata.usgs.gov/nwis/dvstat?&site_no=11274000&agency_cd=USGS&por_11274000_9587=2208812,00060,9587,{start_date},{end_date}&stat_cds=p{percentile}_va&referred_module=sw&format=rdb"

"""
scrape USGS for 90th percentile flow values 
"https://waterdata.usgs.gov/nwis/dvstat?&site_no=11274000&agency_cd=USGS&por_11274000_9587=2208812,00060,9587,1912-05-01,2022-06-05&stat_cds=p90_va&referred_module=sw&format=rdb"
"""

def get_90th_percentile_flow(start_date, end_date, percentile=90):
	url = f"https://waterdata.usgs.gov/nwis/dvstat?&site_no=11274000&agency_cd=USGS&por_11274000_9587=2208812,00060,9587,{start_date},{end_date}&stat_cds=p{percentile}_va&referred_module=sw&format=rdb"
	df = pd.read_csv(url, skiprows=44, sep="\t")
	df = df.drop(index = [0])
	return df

percent_flow = get_90th_percentile_flow(
	start_date="1912-05-01",
	end_date="2022-06-05",
	# start_date="2022-05-01",
	# end_date="2022-06-05",
	percentile=90
	)
date = arrow.now()
percent_flow.pipe(lambda df:df.loc[
	(df['month_nu'] == date.format('M'))
	& (df['day_nu'] == date.format('D'))
	])['p90_va'].iloc[0]

'5290'

In [5]:
import pandas as pd
from tableauscraper import TableauScraper as TS
# url = "https://public.tableau.com/app/profile/lisa.hong6413/viz/shared/K54XMYYQ6"
url ='https://public.tableau.com/shared/Y7D4HBGG2'
ts = TS()
ts.loads(url)

In [37]:
wb = ts.getWorkbook()
wb
for t in wb.getSheets():
    print(f"worksheet name : {t['sheet']}") #show worksheet name
    # print(t.data) #show dataframe for this worksheet

worksheet name : Curtailment List
worksheet name : Dashboard 2
worksheet name : Dashboard 1
worksheet name : Story 1


In [25]:
# <div class='tableauPlaceholder' id='viz1673725811083' style='position: relative'><noscript><a href='https:&#47;&#47;www.waterboards.ca.gov&#47;'><img alt='Story 1 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;K5&#47;K54XMYYQ6&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='views&#47;DeltaWatershedCurtailmentStatusList&#47;Story1?:language=en-US&amp;:embed=true' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;K5&#47;K54XMYYQ6&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1673725811083');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='1180px';vizElement.style.height='987px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In [32]:
[i for i in wb.getSheets()][0]

{'sheet': 'Curtailment List',
 'isDashboard': False,
 'isVisible': True,
 'namesOfSubsheets': [],
 'windowId': '{EF675C0C-C44A-4A99-A8FE-D9D8A71A8398}'}

In [70]:
sheets = ['Curtailment List','Dashboard 1','Dashboard 2','Story 1',]
df = wb.getCsvData(sheets[0])
# [i for i in df['Subwatershed'].unique()]
# [i for i in df['Watershed'].unique()]
curtailment_chowchilla = df.pipe(lambda df: df.loc[df['Subwatershed'] == 'Chowchilla'])
print(curtailment_chowchilla.pipe(lambda df: [i for i in df['Currently Curtailed? (Manual Changes)'].unique()]))
print(curtailment_chowchilla.pipe(lambda df: [i for i in df['Currently Curtailed? (Automatic)'].unique()]))
curtailment_chowchilla

['Not Curtailed']
['Not Curtailed']


,WR ID,Primary Owner,Claimed Priority Year,Currently Curtailed? (Manual Changes),In Legal Delta,Search App ID,Search Owner Name,Subwatershed,Water Right Type,Currently Curtailed? (Automatic),Legal Delta?,Note,Priority Date,Reasons for Changes,Water Availability Calc,Watershed,0,1
283,A003714,JENNIFER E PALMER,1923,Not Curtailed,No,True,True,Chowchilla,Appropriative,Not Curtailed,False,NaN,1923/11/14,NaN,NaN,San Joaquin,1,1
337,A004233,"Lazy K Ranch, LLC",1924,Not Curtailed,No,True,True,Chowchilla,Appropriative,Not Curtailed,False,NaN,1924/09/24,NaN,NaN,San Joaquin,1,1
10811,A011155,RONALD D ADAMEE,1945,Not Curtailed,No,True,True,Chowchilla,Appropriative,Not Curtailed,False,NaN,1945/09/17,NaN,NaN,San Joaquin,1,1
10812,A011259,DENNIS E ALDRIDGE,1946,Not Curtailed,No,True,True,Chowchilla,Appropriative,Not Curtailed,False,NaN,1946/01/10,NaN,NaN,San Joaquin,1,1
10813,A011453,SALLY M. WALLACE,1946,Not Curtailed,No,True,True,Chowchilla,Appropriative,Not Curtailed,False,NaN,1946/07/01,NaN,NaN,San Joaquin,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13815,C002553,WILLIAM H MOFFITT,1967,Not Curtailed,No,True,True,Chowchilla,Stockpond,Not Curtailed,False,NaN,1967/01/01,NaN,NaN,San Joaquin,1,1
13816,C002554,WILLIAM H MOFFITT,1967,Not Curtailed,No,True,True,Chowchilla,Stockpond,Not Curtailed,False,NaN,1967/01/01,NaN,NaN,San Joaquin,1,1
13817,C000911,BRIAN JACOBSEN,1968,Not Curtailed,No,True,True,Chowchilla,Stockpond,Not Curtailed,False,NaN,1968/01/01,NaN,NaN,San Joaquin,1,1
13818,C000912,BRIAN JACOBSEN,1968,Not Curtailed,No,True,True,Chowchilla,Stockpond,Not Curtailed,False,NaN,1968/01/01,NaN,NaN,San Joaquin,1,1


In [ ]:

for sht in sheets:
	try:
		sht = wb.getWorksheet(sht)
		# print(sht.getDownloadableSummaryData())
		print(sht.getColumns())
		# print(sht.getDownloadableUnderlyingData())
		# print(sht.select())
		print(sht.name)
	except Exception as e:
		print(e)

In [9]:
for t in wb.worksheets:
    print(f"worksheet name : {t.name}") #show worksheet name
    print(t.data) #show dataframe for this worksheet

# Water year classification

In [26]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def get_water_year_classification():
	url = "https://cdec.water.ca.gov/reportapp/javareports?name=WSI"
	R = requests.get(url)
	soup = BeautifulSoup(R.text,'html.parser')
	pres = soup.find_all('pre')

	T = pres[0]
	brs = T.string.split('\r\n\r\n')
	# return brs
	table_text =brs[11]
	table = BeautifulSoup(table_text,features="lxml").find_all('p')[0]
	table
	rows = [i.split('   ') for i in table.text.split('\r\n')[2:]]
	df = pd.DataFrame(rows)
	df = df.iloc[:,[0,2,4,6,8,10,12]]
	df.columns = df.iloc[0]
	df = df.iloc[[2,3]]
	df

	WY_index = df.loc[df['Forecast Date'] == 'Jan 1, 2023']['  90%'].iloc[0]
	if float(WY_index) > 2.5:
		print('Water Year Index is above 2.5')
	else:
		print('Water Year Index is below 2.5')

Water Year Index is below 2.5


In [ ]:

# df.pipe(df.loc[df['Forecast Date'] == "Jan 1, 2023"])

# Delta in excess

In [3]:
import requests
from bs4 import BeautifulSoup

def check_delta_excess():
	"""
	check https://water.ca.gov/-/media/DWR-Website/Web-Pages/Programs/State-Water-Project/Operations-And-Maintenance/Files/Operations-Control-Office/Delta-Status-And-Operations/Delta-Operations-Daily-Summary.pdf
	for the Delta Conditions using python
	"""
	url = "https://water.ca.gov/-/media/DWR-Website/Web-Pages/Programs/State-Water-Project/Operations-And-Maintenance/Files/Operations-Control-Office/Delta-Status-And-Operations/Delta-Operations-Daily-Summary.pdf"
	R = requests.get(url)
	soup = BeautifulSoup(R.text,'html.parser')
	return soup
	# pres = soup.find_all('pre')
	# return pres
check_delta_excess()

%PDF-1.7

4 0 obj
(Identity)
endobj
5 0 obj
(Adobe)
endobj
8 0 obj
&lt;&lt;
/Filter /FlateDecode
/Length 102610
/Length1 397840
/Type /Stream
&gt;&gt;
stream
x��}	`T���w�]{��}{��#�lH�
$��p#"(W�D@@�����'/l��z�Z�Z�0���RmU���V�J�*j[k�J����l~%�gw�s��7�;ߙ���{
 �HDX3z�q����h�&gt;�
��~��1c���q��
 
7n�䩕v�A��!P�k���#;.]��r��?5u��SKN�A��Kx���Ӧ�/���
��� ��O�Z^�p�4�ʘ?wʨ���&gt;w�(&lt;��W�=�i�w�
`�o����O����I��@-x|���3VFx��k�� ��s��O���'	T������Z��bx&lt;l����}��S�� �l
�
��z�Χ�¬�f���E
�-���O�[Y��0�Q���9�,:u�Y��Xсm��Z�d�iߑ��\�1m��e��m��,�	 ^q꼳�{gY�X��=u��y���&lt;
����~gީ
��Xp����]��ue��b���O[������L����( o=���ϱ���4 Ýo�0��Kw��郻O��p,F��&lt;�ʈ�c`��&gt;��9�����(K��m�� �?E?�H^l�b� �&amp;k@�t�T�f}��pu$j�E� �V8�x�ĴI��x�h�Sҳ�ǒ*e�H�d�Ą���)��.Ѻ^����Vp����)Y��}p
s�4,{�����b���^EW�n�@O�$t��Meq,���
�ѝ���1����
X+='��!��߀{�:8�wa�G�	kX&lt;�Z�&gt;�	�o�����C��0~�gc����Q���G'���ʞ�-�}iͼ���(t�cS��n"�q�?�*�8�&amp;�g��|��lKG7������e�߀�
0�
